<a href="https://colab.research.google.com/github/Maralmaa1/DIV-tasks/blob/main/SimpleConv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing**

In [1]:
import numpy as np
import math

In [2]:
class XavierInitializer:
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class HeInitializer():
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B

In [3]:
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

class AdaGrad:
    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1
    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB) * layer.dB

Problem 1: Creating a one-dimensional convolutional layer class with a limited number of channels to 1

In [4]:
class SimpleConv1d():

    def forward(self, x, w, b):
        a = []
        for i in range(len(w) - 1):
            a.append((x[i:i+len(w)] @ w) + b[0])
        return np.array(a)
    def backward(self, x, w, da):
        db = np.sum(da)
        dw = []
        for i in range(len(w)):
            dw.append(da @ x[i:i+len(da)])
        dw = np.array(dw)
        dx = []
        new_w = np.insert(w[::-1], 0, 0)
        new_w = np.append(new_w, 0)
        for i in range(len(new_w)-1):
            dx.append(new_w[i:i+len(da)] @ da)
        dx = np.array(dx[::-1])
        return db, dw, dx

Problem 2: Calculation of output size after one-dimensional convolution

In [5]:
def output_size_calculation(n_in, F, P=0, S=1):
        n_out = int((n_in + 2*P - F) / S + 1)
        return n_out

output_size_calculation(n_in=4, F=3)

2

Problem 3: Experiment of one-dimensional convolutional layer in a small array

In [6]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

sc1d = SimpleConv1d()
forward_propagation = sc1d.forward(x, w, b)
print("forward propagation output is {}".format(forward_propagation))

delta_a = np.array([10, 20])
db, dw, dx = sc1d.backward(x, w, delta_a)
print("backpropagation value is: \n{}\n{}\n{}".format(db,dw,dx))

forward propagation output is [35 50]
backpropagation value is: 
30
[ 50  80 110]
[ 30 110 170 140]


Problem 4: Creating a one-dimensional convolutional layer class with no limit on the number of channels

In [7]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [8]:

a = np.zeros([3, output_size_calculation(4,3,0,1)])

for och in range(w.shape[0]):
    for ch in range(w.shape[1]):
        for m in range(a.shape[1]):
            a[och,m] += np.sum(x[ch, m:m+w.shape[2]]* w[och,ch,:])

a += b[:,None]
print("print forward prop:", a)


print forward prop: [[16. 22.]
 [17. 23.]
 [18. 24.]]
